In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
from mprompt.modules.emb_diff_module import EmbDiffModule
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
import notebook_helper
import mprompt.viz
import scipy.special
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from mprompt.methods.m4_evaluate import D5_Validator
import openai
from mprompt.modules.fmri_module import fMRIModule
from pprint import pprint
import joblib
import viz
from mprompt.config import RESULTS_DIR
import torch.cuda
import json
openai.api_key_path = os.path.expanduser('~/.OPENAI_KEY')


huth2016_categories = json.load(open('huth2016clusters.json', 'r'))
r = (pd.read_pickle('../results/results_fmri.pkl')
    .sort_values(by=['top_score_synthetic'], ascending=False))
r['id'] = "('" + r['top_explanation_init_strs'].str.replace(' ', '_').str.slice(stop=20) + "', '" + r['subject'] + "', " + r['module_num'].astype(str) + ")"

In [ ]:
huth2016_categories

### Select voxels

In [ ]:
# manually pick some voxels
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 200):
    display(r.sort_values(by=['top_score_synthetic'], ascending=False)[
        ['top_explanation_init_strs', 'subject', 'module_num', 'top_score_synthetic', 'frac_top_ngrams_module_correct', 'id', 'top_ngrams_module_correct']
    ].round(3).reset_index(drop=True).head(50))


# expls = ['baseball','animals','water','movement','religion','time','technology']
# interesting_expls = ['food', 'numbers', 'physical contact', 'time', 'laughter', 'age', 'clothing']
# voxels = [('movement', 'UTS01',	7), ('numbers', 'UTS03', 55), ('time', 'UTS03', 19), ('relationships', 'UTS01', 21),
        #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
# voxels = [('numbers', 'UTS03', 55), ('time', 'UTS03', 19),
        #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
# voxels = [('movement', 'UTS01',	7),('relationships', 'UTS01', 21) ('passing of time	UTS02	4)]
EXPT_NAME = 'relationships_mar9'
voxels = [('relationships', 'UTS02', 9), ('time', 'UTS02', 4), ('looking or staring', 'UTS03', 57), ('food and drinks', 'UTS01', 52), ('hands and arms', 'UTS01', 46)]

In [ ]:
# put all voxel data into rows DataFrame
rows = []
expls = []
for vox in voxels:
    expl, subj, vox_num = vox
    try:
        rows.append(r[(r.subject == subj) & (r.module_num == vox_num)].iloc[0])
        expls.append(expl)
    except:
        print('skipping', vox)
rows = pd.DataFrame(rows)
rows['expl'] = expls

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 200):
    display(rows[['subject', 'module_num', 'expl', 'top_explanation_init_strs', 'top_ngrams_module_correct']])

### Generate story

In [ ]:
version = 'v2'
prompts = notebook_helper.get_prompts(rows, version)
for p in prompts:
    print(p)
PV = notebook_helper.get_prompt_templates(version)

In [ ]:
paragraphs = mprompt.llm.get_paragraphs(prompts, prefix_first=PV['prefix_first'], prefix_next=PV['prefix_next'])
rows['prompt'] = prompts
rows['paragraph'] = paragraphs
for para in paragraphs:
    pprint(para)

In [ ]:
STORIES_DIR = join(RESULTS_DIR, 'stories')
os.makedirs(STORIES_DIR, exist_ok=True)
joblib.dump(rows, join(STORIES_DIR, f'{EXPT_NAME}_rows.pkl'))

In [ ]:
stories = '''
temporal: travel minute leave date clock hours week rumbling next schedule months month immediately heading waited weeks weekend arrive seconds starting destination hour minutes twice parking trip halfway nights pm promptly
abstract: natural roots delicate exaggerated diverse gentle stronger atmosphere flesh soothing qualities muscular distorted describe strong powerful artificial deeper ecology stem pure sound spreading deep particularly intricate subtle masses expressive weak focus environment influenced hip creating intense sensation folk surroundings
professional: meetings owner worker office rented year business meet home decided visit staying paid bank students house members visiting meeting private staff school estate classroom college apartment hotel attend
visual: yellow fur silver badge garment large gold suit steel colour variety brown uniform cap clothing leather breeches coloured colored skull cotton wig bone wears fielder ribbons skin green stockings black seal breast glove stripes striped feathers jackets colors shafts white wide medium color style blazer shaped cloth tan
violent: lethal instantly breath kill bat painful pause repeat tongue stab trigger sentence breathe die accidentally poison accidental swallow explode bullet reaction kills hit swallowed repeatedly loses
tactile: fingers blade metallic fog melt slow vertical dome edges waves drifting absorb barrel inches flowing thin swirling smooth pinch diameter tops sliding thick gravity breeze depth drops lightly blades hits slowly surface thinner sheets heavier portable pressing needle solid cut thicker soft slight finger melting meters cloudy slowed lighter flow faster layers screens lighting inch clouds slower reach shapes stream layer upwards
communal: schools male community church young society interests bred banker family respected american culture catholic adopted whose teaching african among educated founded children public youth politician protect reputation sons wealthy
mental: asleep knew memories overwhelmed awake anxious uneasy studying moments hadn't learning sadness talked experiences sounded confess senses calm fascinating thoughts answered emptiness reading wake dreaming listening tense hearing experience awe reply exploring replies quiet solitude comforting wished explore happened realised discuss
numeric: four quarter pairs set pound pair five maximum extra half drop card pounds cent overhead deck floors three two per stock each top tie shillings purse ten twenty double sold intervals tables smaller six
emotional: alive nature innocent despised disgrace religion spiritual believes troubled emotion illness influence truth tortured compassion fear deeply speak perceived anger embrace religious emotions weakness human feelings harsh vile profoundly openly remark profound evil admiration believing peaceful christian convinced hatred man's cruel fearful betrayed
social: child son situation pleaded marriage parents arrest daughter victim husband informed charges charged suicide relatives sheriff widow accused met arrested confronted eldest father custody robbery pregnant murder mother guilty confessed calls wife court whom stolen refused married murderer murdered convicted
locational: stadium visitors halls company shops golf scenery architecture rooms gardens athletic lounge spacious space evenings building houses art landscape fields sporting shopping arts purchased annual national center stores pools campus facilities sports activities design uniforms clubhouse teams local
'''
words = stories.split()
top_ngrams_dict = {}
start = 1
for i in range(len(words)):
    if words[i].endswith(':'):
        if i > 1:
            end = i
            top_ngrams_dict[k] = words[start: end]
        k = words[i][:-1]
        start = i + 1
end = i + 1
top_ngrams_dict[k] = words[start: end]
        